# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
import warnings
warnings.filterwarnings('ignore')
import requests
from pandas.io.json import json_normalize

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Create dataframe from Part 1 csv
city = pd.read_csv('output_data/cities.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Take smaller dataframe of 30 cities
city_n = city.head(30)
# Take latitude and Longitudes in location variable
locations = city_n[['Lat', 'Lng']]
# Take humidity as weight
weights = city_n['Humidity']
# create gmaps figure
fig = gmaps.figure()
# add location and weight to gmaps
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights,max_intensity=30, point_radius=3.0))
# show gmaps figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Create conditions for ideal weather
cond_temp_80 = city['Max Temp'] < 80
cond_temp_70 = city['Max Temp'] > 70
cond_wind = city['Wind Speed'] < 10
cond_cloud = city['Cloudiness'] ==0

# Take cities with ideal weather in city_cond dataframe
city_cond = city[(cond_temp_80) & (cond_temp_70) & (cond_wind) & (cond_cloud)]

if len(city_cond)==0:
    print('No row meets ideal weather condition, retaining all rows')
    city_ideal = city.copy()
else:
    print('Dataset with ideal weather condition:') 
    city_ideal = city_cond.copy()
    
city_ideal.head(2)


No row meets ideal weather condition, retaining all rows


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Egersund,58.45,6.00,293.71,95.0,100.0,2.24,NO,1.597764e+09
1,Lebu,-37.62,-73.65,282.15,71.0,73.0,4.50,CL,1.597764e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Take first 200 cities 
city_small = city.head(200)

#Create an empty dataframe: hotel_df to load hotel details
hotel_df = pd.DataFrame()

# Create a new column Restaurant to help us debug records for which Google API didnt found any restaurant
city_small['Restaurant'] = ''

# Create partial strings for Google Places API, store it in variables for convenience
static_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="
static_url2 = "&radius=5000&type=restaurant&key="

# Iterate over each row of city_small df, create a url with latitude, longitude
for i in range(len(city_small)):
    lat = city_small[city_small.index==i]['Lat'].item()
    lng = city_small[city_small.index==1]['Lng'].item()
    url = static_url + str(lat) + ',' + str(lng) + static_url2 + g_key 
    r = requests.get(url)
    
    # Add Try except block to handle errors (where no restaurant is found) gracefully
    try: 
        # load results from Google API into dataframe
        df = json_normalize(r.json()['results'])
        cols = ['business_status','name', 'rating', 'user_ratings_total','geometry.location.lat', 'geometry.location.lng' ]
        df = df[cols].head(1)
        df['City'] = city_small[city_small.index==i]['City'].item()
        df['Country'] = city_small[city_small.index==i]['Country'].item()
    except: 
        city.loc[city.index==i, 'Restaurant'] = 'No Restaurant Found'
        
    hotel_df = pd.concat([df,hotel_df],axis=0, ignore_index=True)

# Rename columns    
hotel_df.rename(columns = {'name': 'Hotel Name', 'geometry.location.lat': 'Lat', 'geometry.location.lng': 'Lng'}, inplace=True)

# Keep only required columns
hotel_df = hotel_df[['Hotel Name', 'Lat', 'Lng', 'City', 'Country']]

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))